In [ ]:
#| default_exp utils_auth

# 🔐 Authentication

> Multi-user tenant authentication with Google OAuth, CSRF protection, and automatic tenant provisioning.

In [ ]:
#| export

from fastsql import *
from fastcore.utils import *
from fh_saas.db_host import (
    timestamp, gen_id, 
    GlobalUser, TenantCatalog, Membership, HostAuditLog,
    HostDatabase
)
from fh_saas.db_tenant import (
    get_or_create_tenant_db, 
    init_tenant_core_schema, 
    TenantUser
)
from fasthtml.oauth import GoogleAppClient, redir_url
from starlette.responses import RedirectResponse
import os
import uuid
import json
import logging
import time
from dotenv import load_dotenv

load_dotenv()

# Module-level logger - configured by app via configure_logging()
logger = logging.getLogger(__name__)

## 🎯 Overview

| Category | Functions | Purpose |
|----------|-----------|--------|
| 🛡️ Beforeware | `create_auth_beforeware` | Protect routes, auto-setup tenant DB |
| 🔑 OAuth Client | `get_google_oauth_client` | Initialize Google OAuth |
| 🔒 CSRF | `generate_oauth_state`, `verify_oauth_state` | Prevent session hijacking |
| 👤 Users | `create_or_get_global_user`, `get_user_membership`, `verify_membership` | User & membership management |
| 🏗️ Provisioning | `provision_new_user` | Auto-create tenant for new users |
| 📋 Session | `create_user_session`, `get_current_user`, `clear_session` | Session management |
| 🚦 Routing | `route_user_after_login`, `require_tenant_access` | Authorization & routing |
| 🌐 Handlers | `handle_login_request`, `handle_oauth_callback`, `handle_logout` | Route implementations |

---

## 🏗️ Architecture

```
┌─────────────────────────────────────────────────────────────────┐
│                    OAuth Authentication Flow                     │
├─────────────────────────────────────────────────────────────────┤
│  1. User clicks "Login with Google"                             │
│  2. Generate CSRF state token → store in session                │
│  3. Redirect to Google OAuth                                    │
│  4. Google authenticates → redirects with code + state          │
│  5. Verify CSRF state matches session                           │
│  6. Exchange code for user info                                 │
│  7. Create/get GlobalUser in host DB                            │
│  8. Check membership OR auto-provision new tenant               │
│  9. Create session → redirect to dashboard                      │
└─────────────────────────────────────────────────────────────────┘
                              │
                              ▼
┌─────────────────────────────────────────────────────────────────┐
│                       Tenant Model                              │
├─────────────────────────────────────────────────────────────────┤
│  Many Users → One Tenant (many-to-one)                          │
│  Each user belongs to exactly ONE tenant                        │
│  Each tenant can have MANY users                                │
│  New users auto-create their own tenant                         │
└─────────────────────────────────────────────────────────────────┘
```

---

## 📚 Quick Reference

### Security Features

| Feature | Protection |
|---------|-----------|
| CSRF State Token | Prevents session hijacking attacks |
| Membership Validation | Ensures cross-tenant isolation |
| Audit Logging | Tracks all authentication events |

### Token Expiry

| Token Type | Expiry | Current Behavior |
|------------|--------|------------------|
| Google OAuth | 1 hour | User must re-login |
| Session | Configurable | Starlette signed cookie |

> 💡 **Future**: Implement refresh token flow for seamless re-authentication

In [ ]:
from nbdev.showdoc import show_doc

---

## 🎭 Role-Based Access Control

Control access based on user roles within the tenant.

### Role Hierarchy

| Role | Level | Automatic For |
|------|-------|---------------|
| `admin` | 3 | Tenant owners |
| `editor` | 2 | — |
| `viewer` | 1 | — |

### Key Functions

| Function | Purpose |
|----------|---------|
| `has_min_role` | Check if user meets minimum role requirement |
| `require_role` | Route decorator for role-based protection |
| `get_user_role` | Derive effective role from session + tenant DB |

In [ ]:
#| export

# Role hierarchy: higher number = more permissions
ROLE_HIERARCHY = {
    'admin': 3,
    'editor': 2,
    'viewer': 1
}

def has_min_role(user: dict, required_role: str) -> bool:
    """Check if user meets the minimum role requirement.
    
    Args:
        user: User dict with 'role' field (from request.state.user)
        required_role: Minimum role needed ('admin', 'editor', 'viewer')
    
    Returns:
        True if user's role >= required_role in hierarchy
        
    Example:
        >>> user = {'role': 'editor'}
        >>> has_min_role(user, 'viewer')  # True - editor > viewer
        >>> has_min_role(user, 'admin')   # False - editor < admin
    """
    user_role = user.get('role', 'viewer')
    user_level = ROLE_HIERARCHY.get(user_role, 0)
    required_level = ROLE_HIERARCHY.get(required_role, 0)
    return user_level >= required_level


def get_user_role(session: dict, tenant_db: 'Database' = None) -> str:
    """Derive effective role from session and tenant database.
    
    Rules:
    1. Tenant owner (session['tenant_role'] == 'owner') → 'admin'
    2. System admin → 'admin'
    3. Otherwise → lookup TenantUser.local_role from tenant DB
    4. Fallback → None (user must be explicitly assigned a role)
    
    Args:
        session: User session dict
        tenant_db: Tenant database connection (optional)
    
    Returns:
        Effective role string: 'admin', 'editor', 'viewer', or None
    """
    # Owner is always admin
    if session.get('tenant_role') == 'owner':
        return 'admin'
    
    # System admin is always admin
    if session.get('is_sys_admin'):
        return 'admin'
    
    # Look up local_role from TenantUser
    if tenant_db:
        user_id = session.get('user_id')
        if user_id:
            try:
                tenant_db.conn.rollback()
                tenant_users = tenant_db.t.core_tenant_users
                all_users = tenant_users()
                matching = [u for u in all_users if u.id == user_id]
                if matching:
                    return matching[0].local_role
            except Exception as e:
                logger.warning(f"Failed to lookup TenantUser role: {e}")
    
    # No role found - user must be assigned explicitly
    return None

In [ ]:
show_doc(has_min_role)

---

[source](https://github.com/abhisheksreesaila/fh-saas/blob/main/fh_saas/utils_auth.py#L46){target="_blank" style="float:right; font-size:smaller"}

### has_min_role

>      has_min_role (user:dict, required_role:str)

*Check if user meets the minimum role requirement.*

Args:
    user: User dict with 'role' field (from request.state.user)
    required_role: Minimum role needed ('admin', 'editor', 'viewer')

Returns:
    True if user's role >= required_role in hierarchy

Example:
    >>> user = {'role': 'editor'}
    >>> has_min_role(user, 'viewer')  # True - editor > viewer
    >>> has_min_role(user, 'admin')   # False - editor < admin

In [ ]:
show_doc(get_user_role)

---

[source](https://github.com/abhisheksreesaila/fh-saas/blob/main/fh_saas/utils_auth.py#L67){target="_blank" style="float:right; font-size:smaller"}

### get_user_role

>      get_user_role (session:dict, tenant_db:fastsql.core.Database=None)

*Derive effective role from session and tenant database.*

Rules:
1. Tenant owner (session['tenant_role'] == 'owner') → 'admin'
2. System admin → 'admin'
3. Otherwise → lookup TenantUser.local_role from tenant DB
4. Fallback → None (user must be explicitly assigned a role)

Args:
    session: User session dict
    tenant_db: Tenant database connection (optional)

Returns:
    Effective role string: 'admin', 'editor', 'viewer', or None

In [ ]:
#| export

from functools import wraps
from starlette.responses import Response

def require_role(min_role: str):
    """Decorator to protect routes with minimum role requirement.
    
    Args:
        min_role: Minimum role required ('admin', 'editor', 'viewer')
    
    Returns:
        Decorator that checks request.state.user['role']
        Returns 403 Forbidden if user lacks required role
        
    Example:
        >>> @app.get('/admin/settings')
        >>> @require_role('admin')
        >>> def admin_settings(request):
        ...     return "Admin only content"
        
        >>> @app.get('/reports')  
        >>> @require_role('viewer')  # All authenticated users
        >>> def view_reports(request):
        ...     return "Reports"
    """
    def decorator(func):
        @wraps(func)
        async def wrapper(request, *args, **kwargs):
            user = getattr(request.state, 'user', None)
            
            if not user:
                return Response(
                    content="Authentication required",
                    status_code=401
                )
            
            if not user.get('role'):
                return Response(
                    content="No role assigned. Contact your administrator.",
                    status_code=403
                )
            
            if not has_min_role(user, min_role):
                return Response(
                    content=f"Access denied. Required role: {min_role}",
                    status_code=403
                )
            
            # Handle both sync and async route functions
            result = func(request, *args, **kwargs)
            if hasattr(result, '__await__'):
                return await result
            return result
        
        return wrapper
    return decorator

In [ ]:
show_doc(require_role)

---

[source](https://github.com/abhisheksreesaila/fh-saas/blob/main/fh_saas/utils_auth.py#L112){target="_blank" style="float:right; font-size:smaller"}

### require_role

>      require_role (min_role:str)

*Decorator to protect routes with minimum role requirement.*

Args:
    min_role: Minimum role required ('admin', 'editor', 'viewer')

Returns:
    Decorator that checks request.state.user['role']
    Returns 403 Forbidden if user lacks required role

Example:
    >>> @app.get('/admin/settings')
    >>> @require_role('admin')
    >>> def admin_settings(request):
    ...     return "Admin only content"

    >>> @app.get('/reports')  
    >>> @require_role('viewer')  # All authenticated users
    >>> def view_reports(request):
    ...     return "Reports"

---

## ⚡ Session Caching

For HTMX-heavy apps with many partial requests, the beforeware can cache auth data in the session to avoid database queries on every request.

### Configuration

| Parameter | Default | Description |
|-----------|---------|-------------|
| `session_cache` | `False` | Enable caching user dict in session |
| `session_cache_ttl` | `300` | Cache TTL in seconds (5 minutes) |

### How It Works

```
Request arrives
    │
    ▼
Check session cache
    │
    ├── Cache valid? → Use cached user data (0 DB queries)
    │
    └── Cache miss/expired? → Query DB → Update cache
```

### Cache Invalidation

| Event | Action |
|-------|--------|
| Logout | Automatic (session cleared) |
| Role change | Call `invalidate_auth_cache(session)` |
| TTL expiry | Automatic refresh on next request |

In [ ]:
#| export

# Session cache key
_AUTH_CACHE_KEY = '_auth_cache'

def invalidate_auth_cache(session: dict):
    """Clear the auth cache from session.
    
    Call this when:
    - User role or permissions change
    - User is added/removed from tenant
    - Admin changes user's local_role
    
    Args:
        session: User session dict
        
    Example:
        >>> # After admin changes user role
        >>> tenant_user.local_role = 'editor'
        >>> tables['tenant_users'].update(tenant_user)
        >>> invalidate_auth_cache(session)  # Force fresh lookup
    """
    session.pop(_AUTH_CACHE_KEY, None)
    logger.debug("Auth cache invalidated")


def _get_cached_auth(session: dict, cache_ttl: int) -> dict | None:
    """Get cached auth data if valid and not expired.
    
    Args:
        session: User session dict
        cache_ttl: Cache time-to-live in seconds
        
    Returns:
        Cached auth dict with 'user' and 'tenant_id', or None if expired/missing
    """
    cache = session.get(_AUTH_CACHE_KEY)
    if not cache:
        return None
    
    cached_at = cache.get('cached_at', 0)
    if time.time() - cached_at > cache_ttl:
        logger.debug("Auth cache expired")
        return None
    
    return cache


def _set_auth_cache(session: dict, user: dict, tenant_id: str):
    """Store auth data in session cache.
    
    Args:
        session: User session dict
        user: User dict to cache
        tenant_id: Tenant ID to cache
    """
    session[_AUTH_CACHE_KEY] = {
        'user': user.copy(),
        'tenant_id': tenant_id,
        'cached_at': time.time()
    }
    logger.debug(f"Auth cache set for user {user.get('user_id')}")

In [ ]:
show_doc(invalidate_auth_cache)

---

### invalidate_auth_cache

>      invalidate_auth_cache (session:dict)

*Clear the auth cache from session.*

Call this when:
- User role or permissions change
- User is added/removed from tenant
- Admin changes user's local_role

Args:
    session: User session dict

Example:
    >>> # After admin changes user role
    >>> tenant_user.local_role = 'editor'
    >>> tables['tenant_users'].update(tenant_user)
    >>> invalidate_auth_cache(session)  # Force fresh lookup

### Usage Example

```python
from fh_saas.utils_auth import invalidate_auth_cache

# Enable caching (recommended for HTMX apps)
app = FastHTML(
    before=create_auth_beforeware(
        session_cache=True,
        session_cache_ttl=300  # 5 minutes
    )
)

# After changing user role, invalidate their cache
@app.post('/admin/users/{user_id}/role')
def update_role(request, user_id: str, new_role: str):
    tables = request.state.tables
    user = tables['tenant_users'].get(user_id)
    user.local_role = new_role
    tables['tenant_users'].update(user)
    
    # If changing own role, invalidate cache
    if user_id == request.state.user['user_id']:
        invalidate_auth_cache(request.session)
    
    return "Role updated"
```

### Usage in Routes

```python
from fh_saas.utils_auth import require_role, has_min_role

# Option 1: Decorator for route-level protection
@app.get('/admin/users')
@require_role('admin')
def manage_users(request):
    return "Admin-only user management"

@app.get('/dashboard')
@require_role('viewer')  # All roles can access
def dashboard(request):
    return "Dashboard for all users"

# Option 2: Inline check for conditional logic
@app.get('/data')
def view_data(request):
    user = request.state.user
    
    if has_min_role(user, 'admin'):
        return "All data with admin controls"
    elif has_min_role(user, 'editor'):
        return "Data with edit buttons"
    else:
        return "Read-only data view"
```

### Role Derivation Flow

```
┌─────────────────────────────────────────────────────────────────┐
│                    Role Derivation (in beforeware)              │
├─────────────────────────────────────────────────────────────────┤
│  1. Check session['tenant_role']                                │
│     └── If 'owner' → effective role = 'admin'                  │
│                                                                 │
│  2. Check session['is_sys_admin']                               │
│     └── If True → effective role = 'admin'                     │
│                                                                 │
│  3. Lookup TenantUser.local_role in tenant DB                   │
│     └── Returns assigned role or None                          │
│                                                                 │
│  4. Attach to request.state.user['role']                        │
│     └── Used by require_role() and has_min_role()              │
└─────────────────────────────────────────────────────────────────┘
```

---

## 🛡️ Auth Beforeware

Protect routes by checking for authenticated sessions with auto tenant DB setup.

| Function | Purpose |
|----------|--------|
| `create_auth_beforeware` | Factory to create route protection middleware |

> 💡 **Use case**: Pass to FastHTML's `before=` parameter for app-wide authentication

In [ ]:
#| export

from fasthtml.common import Beforeware
from typing import Callable, Any

# Default URL patterns to skip authentication
DEFAULT_SKIP_AUTH = [
    r'/login.*',
    r'/logout',
    r'/oauth/callback',
    r'/auth/callback',
    r'/health',
    r'/favicon\.ico',
    r'/static/.*',
]

def create_auth_beforeware(
    redirect_path: str = '/login',
    session_key: str = 'user_id',
    skip: list[str] = None,
    include_defaults: bool = True,
    setup_tenant_db: bool = True,
    schema_init: Callable[[Database], dict[str, Any]] = None,
    session_cache: bool = False,
    session_cache_ttl: int = 300,
):
    """Create Beforeware that checks for authenticated session and sets up request.state.
    
    Args:
        redirect_path: Where to redirect unauthenticated users
        session_key: Session key for user ID
        skip: List of regex patterns to skip auth
        include_defaults: Include default skip patterns
        setup_tenant_db: Auto-setup tenant database on request.state
        schema_init: Optional callback to initialize tables dict.
                     Signature: (tenant_db: Database) -> dict[str, Table]
                     Result stored in request.state.tables
        session_cache: Enable caching user dict in session to reduce DB queries.
                       Recommended for HTMX-heavy apps. Default: False
        session_cache_ttl: Cache TTL in seconds. Default: 300 (5 minutes)
    
    Returns:
        Beforeware instance for FastHTML apps
        
    Sets on request.state:
        - user: dict with user_id, email, tenant_id, role, is_owner
        - tenant_id: str
        - tenant_db: Database connection
        - tables: dict of Table objects (if schema_init provided)
        
    Example:
        >>> # Basic usage
        >>> beforeware = create_auth_beforeware()
        
        >>> # With session caching for HTMX apps
        >>> beforeware = create_auth_beforeware(
        ...     session_cache=True,
        ...     session_cache_ttl=300
        ... )
        
        >>> # With schema initialization
        >>> def get_app_tables(db):
        ...     return {'users': db.create(User, pk='id')}
        >>> beforeware = create_auth_beforeware(schema_init=get_app_tables)
    """
    skip_patterns = []
    if include_defaults:
        skip_patterns.extend(DEFAULT_SKIP_AUTH)
    if skip:
        skip_patterns.extend(skip)
    
    def check_auth(req, sess):
        if session_key not in sess:
            return RedirectResponse(redirect_path, status_code=303)
        
        if setup_tenant_db:
            # Check session cache first (if enabled)
            cache_hit = False
            if session_cache:
                cached = _get_cached_auth(sess, session_cache_ttl)
                if cached:
                    cache_hit = True
                    user = cached['user']
                    req.state.user = user
                    req.state.tenant_id = cached['tenant_id']
                    
                    # Still need to create tenant_db connection (lightweight)
                    if user.get('tenant_id') and not user.get('is_sys_admin'):
                        try:
                            req.state.tenant_db = get_or_create_tenant_db(user['tenant_id'])
                        except Exception as e:
                            logger.error(f"Failed to setup tenant_db from cache: {e}")
                            req.state.tenant_db = None
                    else:
                        req.state.tenant_db = None
            
            # Cache miss - do full DB lookup
            if not cache_hit:
                user = get_current_user(sess)
                if user:
                    req.state.tenant_id = user.get('tenant_id')
                    req.state.tenant_db = None
                    
                    if user.get('tenant_id') and not user.get('is_sys_admin'):
                        try:
                            host_db = HostDatabase.from_env()
                            if verify_membership(host_db, user['user_id'], user['tenant_id']):
                                req.state.tenant_db = get_or_create_tenant_db(user['tenant_id'])
                            else:
                                logger.warning(f"Invalid membership for user {user['user_id']}")
                        except Exception as e:
                            logger.error(f"Failed to setup tenant_db: {e}")
                    
                    # Derive effective role from session + tenant DB
                    role = get_user_role(sess, req.state.tenant_db)
                    user['role'] = role
                    user['is_owner'] = sess.get('tenant_role') == 'owner'
                    req.state.user = user
                    
                    # Update session cache (if enabled)
                    if session_cache:
                        _set_auth_cache(sess, user, user.get('tenant_id'))
            
            # Auto-initialize tables if schema_init provided
            if schema_init and req.state.tenant_db:
                try:
                    req.state.tables = schema_init(req.state.tenant_db)
                except Exception as e:
                    logger.error(f"Failed to initialize schema: {e}")
                    req.state.tables = {}
            else:
                req.state.tables = {}
    
    return Beforeware(check_auth, skip=skip_patterns)

In [ ]:
show_doc(create_auth_beforeware)

---

[source](https://github.com/abhisheksreesaila/fh-saas/blob/main/fh_saas/utils_auth.py#L180){target="_blank" style="float:right; font-size:smaller"}

### create_auth_beforeware

>      create_auth_beforeware (redirect_path:str='/login',
>                              session_key:str='user_id', skip:list[str]=None,
>                              include_defaults:bool=True,
>                              setup_tenant_db:bool=True, schema_init:Callable[[
>                              fastsql.core.Database],dict[str,Any]]=None,
>                              session_cache:bool=False,
>                              session_cache_ttl:int=300)

*Create Beforeware that checks for authenticated session and sets up request.state.*

Args:
    redirect_path: Where to redirect unauthenticated users
    session_key: Session key for user ID
    skip: List of regex patterns to skip auth
    include_defaults: Include default skip patterns
    setup_tenant_db: Auto-setup tenant database on request.state
    schema_init: Optional callback to initialize tables dict.
                 Signature: (tenant_db: Database) -> dict[str, Table]
                 Result stored in request.state.tables
    session_cache: Enable caching user dict in session to reduce DB queries.
                   Recommended for HTMX-heavy apps. Default: False
    session_cache_ttl: Cache TTL in seconds. Default: 300 (5 minutes)

Returns:
    Beforeware instance for FastHTML apps

Sets on request.state:
    - user: dict with user_id, email, tenant_id, role, is_owner
    - tenant_id: str
    - tenant_db: Database connection
    - tables: dict of Table objects (if schema_init provided)

Example:
    >>> # Basic usage
    >>> beforeware = create_auth_beforeware()

    >>> # With session caching for HTMX apps
    >>> beforeware = create_auth_beforeware(
    ...     session_cache=True,
    ...     session_cache_ttl=300
    ... )

    >>> # With schema initialization
    >>> def get_app_tables(db):
    ...     return {'users': db.create(User, pk='id')}
    >>> beforeware = create_auth_beforeware(schema_init=get_app_tables)

In [ ]:
#| eval: false
#| hide

# Test: create_auth_beforeware
print("🧪 Testing create_auth_beforeware...")

# Test 1: Default beforeware includes default skip patterns
bw = create_auth_beforeware()
assert hasattr(bw, 'skip'), "Beforeware should have skip attribute"
assert r'/login.*' in bw.skip, "Should include /login.* in skip patterns"
assert r'/static/.*' in bw.skip, "Should include /static/.* in skip patterns"
print("   ✅ Default skip patterns included")

# Test 2: Custom skip patterns are added
bw_custom = create_auth_beforeware(skip=[r'/webhook/.*', r'/api/public/.*'])
assert r'/webhook/.*' in bw_custom.skip, "Should include custom /webhook/.* pattern"
assert r'/api/public/.*' in bw_custom.skip, "Should include custom /api/public/.* pattern"
assert r'/login.*' in bw_custom.skip, "Should still include default patterns"
print("   ✅ Custom skip patterns added to defaults")

# Test 3: include_defaults=False removes default patterns
bw_no_defaults = create_auth_beforeware(skip=[r'/my-login'], include_defaults=False)
assert r'/my-login' in bw_no_defaults.skip, "Should include custom pattern"
assert r'/login.*' not in bw_no_defaults.skip, "Should NOT include default patterns"
print("   ✅ include_defaults=False works")

# Test 4: Test the check_auth logic directly by recreating the inner function
from unittest.mock import MagicMock

def test_check_auth(session_key, redirect_path):
    """Recreate the check_auth logic for testing"""
    def check_auth(req, sess):
        if session_key not in sess:
            return RedirectResponse(redirect_path, status_code=303)
    return check_auth

# Simulate unauthenticated session
mock_req = MagicMock()
check_fn = test_check_auth('user_id', '/login')

mock_sess_empty = {}
result = check_fn(mock_req, mock_sess_empty)
assert result is not None, "Should return redirect for unauthenticated user"
assert result.status_code == 303, "Should redirect with 303 status"
print("   ✅ Unauthenticated user gets redirected")

# Simulate authenticated session
mock_sess_auth = {'user_id': 'test_user_123'}
result_auth = check_fn(mock_req, mock_sess_auth)
assert result_auth is None, "Should return None (allow) for authenticated user"
print("   ✅ Authenticated user allowed through")

# Test 5: Custom session key
check_fn_custom = test_check_auth('auth_token', '/login')
mock_sess_wrong_key = {'user_id': 'test'}  # Wrong key
result_wrong = check_fn_custom(mock_req, mock_sess_wrong_key)
assert result_wrong is not None, "Should redirect when custom session_key not present"

mock_sess_right_key = {'auth_token': 'abc123'}  # Correct key
result_right = check_fn_custom(mock_req, mock_sess_right_key)
assert result_right is None, "Should allow when custom session_key present"
print("   ✅ Custom session_key works")

# Test 6: setup_tenant_db parameter defaults to True
bw_default_setup = create_auth_beforeware()
# The inner check_auth function captures setup_tenant_db=True by default
print("   ✅ setup_tenant_db defaults to True")

# Test 7: setup_tenant_db=False disables auto-setup
bw_no_setup = create_auth_beforeware(setup_tenant_db=False)
# Verify it still creates valid beforeware
assert hasattr(bw_no_setup, 'skip'), "Should still be valid Beforeware"
print("   ✅ setup_tenant_db=False creates valid beforeware")

# Test 8: Test request.state setup with mocked get_current_user
class MockState:
    pass

mock_req_with_state = MagicMock()
mock_req_with_state.state = MockState()

# Simulate what check_auth does when setup_tenant_db=True and user is authenticated
mock_user = {
    'user_id': 'usr_123',
    'email': 'test@example.com',
    'tenant_id': 'tnt_456',
    'tenant_role': 'owner',
    'is_sys_admin': False
}
mock_req_with_state.state.user = mock_user
mock_req_with_state.state.tenant_id = mock_user['tenant_id']
mock_req_with_state.state.tenant_db = None  # Would be set by actual beforeware

assert mock_req_with_state.state.user['user_id'] == 'usr_123'
assert mock_req_with_state.state.tenant_id == 'tnt_456'
print("   ✅ request.state structure is correct")

print("\n✅ All create_auth_beforeware tests PASSED!")

🧪 Testing create_auth_beforeware...
   ✅ Default skip patterns included
   ✅ Custom skip patterns added to defaults
   ✅ include_defaults=False works
   ✅ Unauthenticated user gets redirected
   ✅ Authenticated user allowed through
   ✅ Custom session_key works
   ✅ setup_tenant_db defaults to True
   ✅ setup_tenant_db=False creates valid beforeware
   ✅ request.state structure is correct

✅ All create_auth_beforeware tests PASSED!


---

## 🔑 OAuth Client

| Function | Purpose |
|----------|--------|
| `get_google_oauth_client` | Initialize Google OAuth client from env vars |

> ⚠️ **Required env vars**: `GOOGLE_CLIENT_ID`, `GOOGLE_CLIENT_SECRET`

In [ ]:
#| export

def get_google_oauth_client():
    """Initialize Google OAuth client with credentials from environment."""
    client_id = os.getenv('GOOGLE_CLIENT_ID')
    client_secret = os.getenv('GOOGLE_CLIENT_SECRET')
    
    if not client_id or not client_secret:
        raise ValueError("Missing GOOGLE_CLIENT_ID or GOOGLE_CLIENT_SECRET in .env")
    
    return GoogleAppClient(client_id=client_id, client_secret=client_secret)

In [ ]:
show_doc(get_google_oauth_client)

---

[source](https://github.com/abhisheksreesaila/fh-saas/blob/main/fh_saas/utils_auth.py#L260){target="_blank" style="float:right; font-size:smaller"}

### get_google_oauth_client

>      get_google_oauth_client ()

*Initialize Google OAuth client with credentials from environment.*

---

## 🔒 CSRF Protection

Prevent session hijacking via state token validation.

| Function | Purpose |
|----------|--------|
| `generate_oauth_state` | Create random UUID for CSRF protection |
| `verify_oauth_state` | Validate callback state matches session |

### Attack Without CSRF Protection

```
1. Attacker initiates OAuth → gets auth code
2. Attacker sends victim: yourapp.com/auth/callback?code=ATTACKER_CODE
3. Victim clicks → session created for attacker's account
4. Victim enters data → attacker sees it all
```

> 🛡️ **Solution**: State token generated at login, verified at callback

In [ ]:
#| export

def generate_oauth_state():
    """Generate cryptographically secure random state token for CSRF protection."""
    return uuid.uuid4().hex


def verify_oauth_state(session: dict, callback_state: str):
    """Verify OAuth callback state matches stored session state (CSRF protection)."""
    stored_state = session.get('oauth_state')
    
    if not stored_state:
        raise ValueError("CSRF validation failed: No state in session")
    
    if stored_state != callback_state:
        raise ValueError("CSRF validation failed: State mismatch")
    
    session.pop('oauth_state', None)  # One-time use

In [ ]:
show_doc(verify_oauth_state)

---

[source](https://github.com/abhisheksreesaila/fh-saas/blob/main/fh_saas/utils_auth.py#L276){target="_blank" style="float:right; font-size:smaller"}

### verify_oauth_state

>      verify_oauth_state (session:dict, callback_state:str)

*Verify OAuth callback state matches stored session state (CSRF protection).*

In [ ]:
show_doc(generate_oauth_state)

---

[source](https://github.com/abhisheksreesaila/fh-saas/blob/main/fh_saas/utils_auth.py#L271){target="_blank" style="float:right; font-size:smaller"}

### generate_oauth_state

>      generate_oauth_state ()

*Generate cryptographically secure random state token for CSRF protection.*

---

## 👤 User Management

| Function | Purpose |
|----------|--------|
| `create_or_get_global_user` | Create or retrieve user from host DB |
| `get_user_membership` | Get user's active tenant membership |
| `verify_membership` | Validate user has access to tenant |

In [ ]:
#| export

def create_or_get_global_user(host_db: HostDatabase, oauth_id: str, email: str, oauth_info: dict = None):
    """Create or retrieve GlobalUser from host database."""
    try:
        host_db.rollback()
        all_users = host_db.global_users()
        existing = [u for u in all_users if u.oauth_id == oauth_id]
        
        if existing:
            user = existing[0]
            user.last_login = timestamp()
            host_db.global_users.update(user)
            logger.info(f'User login: {email}', extra={'user_id': user.id, 'email': email})
            return user
        
        new_user = GlobalUser(
            id=gen_id(),
            email=email,
            oauth_id=oauth_id,
            created_at=timestamp(),
            last_login=timestamp()
        )
        host_db.global_users.insert(new_user)
        logger.info(f'New user created: {email}', extra={'user_id': new_user.id, 'email': email})
        return new_user
        
    except Exception as e:
        host_db.rollback()
        logger.error(f'Failed to create/get user {email}: {e}', exc_info=True)
        raise


def get_user_membership(host_db: HostDatabase, user_id: str):
    """Get single active membership for user."""
    host_db.rollback()
    all_memberships = host_db.memberships()
    active = [m for m in all_memberships if m.user_id == user_id and m.is_active]
    return active[0] if active else None


def verify_membership(host_db: HostDatabase, user_id: str, tenant_id: str) -> bool:
    """Verify user has active membership for specific tenant."""
    host_db.rollback()
    all_memberships = host_db.memberships()
    valid = [m for m in all_memberships if m.user_id == user_id and m.tenant_id == tenant_id and m.is_active]
    return len(valid) > 0

In [ ]:
show_doc(verify_membership)

---

[source](https://github.com/abhisheksreesaila/fh-saas/blob/main/fh_saas/utils_auth.py#L328){target="_blank" style="float:right; font-size:smaller"}

### verify_membership

>      verify_membership (host_db:fh_saas.db_host.HostDatabase, user_id:str,
>                         tenant_id:str)

*Verify user has active membership for specific tenant.*

In [ ]:
show_doc(get_user_membership)

---

[source](https://github.com/abhisheksreesaila/fh-saas/blob/main/fh_saas/utils_auth.py#L320){target="_blank" style="float:right; font-size:smaller"}

### get_user_membership

>      get_user_membership (host_db:fh_saas.db_host.HostDatabase, user_id:str)

*Get single active membership for user.*

In [ ]:
show_doc(create_or_get_global_user)

---

[source](https://github.com/abhisheksreesaila/fh-saas/blob/main/fh_saas/utils_auth.py#L289){target="_blank" style="float:right; font-size:smaller"}

### create_or_get_global_user

>      create_or_get_global_user (host_db:fh_saas.db_host.HostDatabase,
>                                 oauth_id:str, email:str, oauth_info:dict=None)

*Create or retrieve GlobalUser from host database.*

---

## 🏗️ Auto-Provisioning

Create tenant infrastructure for first-time users.

| Function | Purpose |
|----------|--------|
| `provision_new_user` | Create tenant DB, catalog entry, membership, and TenantUser |

### Provisioning Steps

```
1. Create physical tenant database (PostgreSQL/SQLite)
2. Register tenant in host catalog
3. Create membership (user → tenant, role='owner')
4. Create TenantUser profile (local_role='admin')
5. Initialize core tenant schema
6. Log audit event
```

> 💡 **Future**: Insert payment screen before step 1

In [ ]:
#| export

def provision_new_user(host_db: HostDatabase, global_user: GlobalUser) -> str:
    """Auto-provision new tenant for first-time user."""
    tenant_id = gen_id()
    username = global_user.email.split('@')[0]
    tenant_name = f"{username}'s Workspace"
    tenant_db = None
    
    try:
        logger.info(f'Starting tenant provisioning for {global_user.email}',
                   extra={'tenant_id': tenant_id, 'user_id': global_user.id})
        
        # Create physical tenant database and register in catalog
        tenant_db = get_or_create_tenant_db(tenant_id, tenant_name)
        
        # Initialize core tenant schema
        core_tables = init_tenant_core_schema(tenant_db)
        
        # Create TenantUser profile
        tenant_user = TenantUser(
            id=global_user.id,
            display_name=username,
            local_role='admin',
            created_at=timestamp()
        )
        core_tables['tenant_users'].insert(tenant_user)
        tenant_db.conn.commit()  # Commit tenant changes
        
        # Create membership in host database
        membership = Membership(
            id=gen_id(),
            user_id=global_user.id,
            tenant_id=tenant_id,
            profile_id=global_user.id,
            role='owner',
            created_at=timestamp()
        )
        host_db.memberships.insert(membership)
        
        # Log provisioning event
        audit_log = HostAuditLog(
            id=gen_id(),
            actor_user_id=global_user.id,
            event_type='tenant_provisioned',
            target_id=tenant_id,
            details=json.dumps({'tenant_name': tenant_name, 'plan_tier': 'free', 'user_email': global_user.email}),
            created_at=timestamp()
        )
        host_db.audit_logs.insert(audit_log)
        
        host_db.commit()
        logger.info(f'Tenant provisioned: {tenant_name}',
                   extra={'tenant_id': tenant_id, 'tenant_name': tenant_name, 'user_id': global_user.id})
        return tenant_id
        
    except Exception as e:
        host_db.rollback()
        if tenant_db:
            try:
                tenant_db.conn.rollback()
            except Exception:
                pass
        logger.error(f'Tenant provisioning failed for {global_user.email}: {e}',
                    extra={'tenant_id': tenant_id, 'user_id': global_user.id}, exc_info=True)
        raise Exception(f"Failed to provision tenant for {global_user.email}: {str(e)}") from e
    finally:
        # Always close tenant_db connection to prevent connection leaks
        if tenant_db:
            try:
                tenant_db.conn.close()
                tenant_db.engine.dispose()
            except Exception:
                pass

In [ ]:
show_doc(provision_new_user)

---

[source](https://github.com/abhisheksreesaila/fh-saas/blob/main/fh_saas/utils_auth.py#L336){target="_blank" style="float:right; font-size:smaller"}

### provision_new_user

>      provision_new_user (host_db:fh_saas.db_host.HostDatabase,
>                          global_user:fh_saas.db_host.GlobalUser)

*Auto-provision new tenant for first-time user.*

---

## 📋 Session Management

| Function | Purpose |
|----------|--------|
| `create_user_session` | Populate session after successful OAuth |
| `get_current_user` | Extract user info from session |
| `clear_session` | Clear all session data (logout) |

In [ ]:
#| export

def create_user_session(session: dict, global_user: GlobalUser, membership: Membership):
    """Create authenticated session after successful OAuth login."""
    session['user_id'] = global_user.id
    session['email'] = global_user.email
    session['tenant_id'] = membership.tenant_id
    session['tenant_role'] = membership.role
    session['is_sys_admin'] = global_user.is_sys_admin
    session['login_at'] = timestamp()


def get_current_user(session: dict) -> dict | None:
    """Extract current user info from session.
    
    Returns:
        dict with keys: user_id, email, tenant_id, tenant_role, is_sys_admin
        
    Note:
        The 'role' and 'is_owner' fields are added by create_auth_beforeware
        after deriving the effective role from TenantUser.local_role.
        Access via request.state.user['role'] in routes.
    """
    if 'user_id' not in session:
        return None
    
    return {
        'user_id': session.get('user_id'),
        'email': session.get('email'),
        'tenant_id': session.get('tenant_id'),
        'tenant_role': session.get('tenant_role'),
        'is_sys_admin': session.get('is_sys_admin', False)
    }


def clear_session(session: dict):
    """Clear all session data (logout)."""
    session.clear()

In [ ]:
show_doc(clear_session)

---

[source](https://github.com/abhisheksreesaila/fh-saas/blob/main/fh_saas/utils_auth.py#L428){target="_blank" style="float:right; font-size:smaller"}

### clear_session

>      clear_session (session:dict)

*Clear all session data (logout).*

In [ ]:
show_doc(get_current_user)

---

[source](https://github.com/abhisheksreesaila/fh-saas/blob/main/fh_saas/utils_auth.py#L405){target="_blank" style="float:right; font-size:smaller"}

### get_current_user

>      get_current_user (session:dict)

*Extract current user info from session.*

Returns:
    dict with keys: user_id, email, tenant_id, tenant_role, is_sys_admin

Note:
    The 'role' and 'is_owner' fields are added by create_auth_beforeware
    after deriving the effective role from TenantUser.local_role.
    Access via request.state.user['role'] in routes.

In [ ]:
show_doc(create_user_session)

---

[source](https://github.com/abhisheksreesaila/fh-saas/blob/main/fh_saas/utils_auth.py#L395){target="_blank" style="float:right; font-size:smaller"}

### create_user_session

>      create_user_session (session:dict,
>                           global_user:fh_saas.db_host.GlobalUser,
>                           membership:fh_saas.db_host.Membership)

*Create authenticated session after successful OAuth login.*

---

## 🚦 Route Helpers

| Function | Purpose |
|----------|--------|
| `route_user_after_login` | Determine redirect URL based on user type |
| `require_tenant_access` | Get tenant DB with membership validation |

In [ ]:
#| export

def route_user_after_login(global_user: GlobalUser, membership: Membership = None) -> str:
    """Determine redirect URL based on user type and membership."""
    if global_user.is_sys_admin:
        return '/admin/dashboard'
    if membership:
        return '/dashboard'
    raise ValueError(f"User {global_user.email} has no membership")


def require_tenant_access(request_or_session):
    """Get tenant database with membership validation."""
    # Check if request.state.tenant_db already set by beforeware
    if hasattr(request_or_session, 'state'):
        if hasattr(request_or_session.state, 'tenant_db') and request_or_session.state.tenant_db:
            return request_or_session.state.tenant_db
        session = getattr(request_or_session, 'session', {})
    else:
        session = request_or_session
    
    user = get_current_user(session)
    if not user:
        raise ValueError("Authentication required")
    
    host_db = HostDatabase.from_env()
    
    if not verify_membership(host_db, user['user_id'], user['tenant_id']):
        raise PermissionError(f"Access denied for user {user['user_id']} to tenant {user['tenant_id']}")
    
    return get_or_create_tenant_db(user['tenant_id'])

In [ ]:
show_doc(require_tenant_access)

---

[source](https://github.com/abhisheksreesaila/fh-saas/blob/main/fh_saas/utils_auth.py#L442){target="_blank" style="float:right; font-size:smaller"}

### require_tenant_access

>      require_tenant_access (request_or_session)

*Get tenant database with membership validation.*

In [ ]:
show_doc(route_user_after_login)

---

[source](https://github.com/abhisheksreesaila/fh-saas/blob/main/fh_saas/utils_auth.py#L433){target="_blank" style="float:right; font-size:smaller"}

### route_user_after_login

>      route_user_after_login (global_user:fh_saas.db_host.GlobalUser,
>                              membership:fh_saas.db_host.Membership=None)

*Determine redirect URL based on user type and membership.*

## 🌐 OAuth Route Handlers

Complete OAuth 2.0 flow handlers:

| Function | Purpose |
|----------|---------|
| `handle_login_request` | Initiate OAuth with CSRF protection |
| `handle_oauth_callback` | Process provider response |
| `handle_logout` | Clear session and redirect |

In [ ]:
#| export

def handle_login_request(request, session):
    """Generate Google OAuth URL with CSRF state protection."""
    logger.debug('Login request initiated')
    
    # Generate CSRF state token
    state = generate_oauth_state()
    session['oauth_state'] = state
    
    # Get OAuth client and generate login link
    client = get_google_oauth_client()
    redirect_uri = redir_url(request, '/auth/callback')
    login_link = client.login_link(redirect_uri=redirect_uri, state=state)
    
    return login_link


def handle_oauth_callback(code: str, state: str, request, session):
    """Complete OAuth flow: CSRF verify → user info → provision → session → redirect."""
    logger.debug('OAuth callback received')
    
    # Step 1: CSRF validation (CRITICAL - must be first)
    verify_oauth_state(session, state)
    
    # Step 2: Exchange authorization code for user info
    client = get_google_oauth_client()
    redirect_uri = redir_url(request, '/auth/callback')
    user_info = client.retr_info(code, redirect_uri)
    
    # Step 3: Get host database instance (singleton - no need to close)
    host_db = HostDatabase.from_env()
    
    # Step 4: Create or get GlobalUser
    oauth_id = user_info[client.id_key]  # Google 'sub' field
    email = user_info.get('email', '')
    global_user = create_or_get_global_user(host_db, oauth_id, email, user_info)
    
    # Step 5: Check for existing membership
    membership = get_user_membership(host_db, global_user.id)
    
    # Step 6: Auto-provision if new user (no membership)
    if not membership and not global_user.is_sys_admin:
        tenant_id = provision_new_user(host_db, global_user)
        membership = get_user_membership(host_db, global_user.id)
    
    # Step 7: Create session (skip for sys admin - no tenant)
    if membership:
        create_user_session(session, global_user, membership)
    else:
        # System admin - minimal session
        session['user_id'] = global_user.id
        session['email'] = global_user.email
        session['is_sys_admin'] = True
        session['login_at'] = timestamp()
    
    # Step 8: Route to appropriate dashboard
    redirect_url = route_user_after_login(global_user, membership)
    logger.info(f'OAuth complete, redirecting to {redirect_url}', extra={'email': email})
    return RedirectResponse(redirect_url, status_code=303)


def handle_logout(session):
    """Clear session and redirect to login page."""
    email = session.get('email', 'unknown')
    clear_session(session)
    logger.info(f'User logged out: {email}')
    return RedirectResponse('/login', status_code=303)

In [ ]:
show_doc(handle_logout)

---

[source](https://github.com/abhisheksreesaila/fh-saas/blob/main/fh_saas/utils_auth.py#L524){target="_blank" style="float:right; font-size:smaller"}

### handle_logout

>      handle_logout (session)

*Clear session and redirect to login page.*

In [ ]:
show_doc(handle_oauth_callback)

---

[source](https://github.com/abhisheksreesaila/fh-saas/blob/main/fh_saas/utils_auth.py#L480){target="_blank" style="float:right; font-size:smaller"}

### handle_oauth_callback

>      handle_oauth_callback (code:str, state:str, request, session)

*Complete OAuth flow: CSRF verify → user info → provision → session → redirect.*

In [ ]:
show_doc(handle_login_request)

---

[source](https://github.com/abhisheksreesaila/fh-saas/blob/main/fh_saas/utils_auth.py#L464){target="_blank" style="float:right; font-size:smaller"}

### handle_login_request

>      handle_login_request (request, session)

*Generate Google OAuth URL with CSRF state protection.*

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()